In [82]:
"""
Hackathon Relâmpago - Caçadores de Fraudes
Autores: João Gabriel Bento & Bruno Lopes
Data: 06/04/2025
Objetivo: Detectar fraudes em lista de compras públicas utilizando Python.
"""


import pandas as pd
from scipy import stats

compras = pd.read_csv('public_servant_purchases_new.csv')

def detectar_compras_duplicadas(compras):
  return compras[compras.duplicated(subset=['item_comprado', 'valor_em_real'])]

In [114]:
def verificar_valores_suspeitos(compras):
  z_scores = stats.zscore(compras['valor_em_real'])

  return compras[(abs(z_scores) > 3) | (compras['valor_em_real'] < 0)]

def compras_sem_valores_suspeitos(compras):
  z_scores = stats.zscore(compras['valor_em_real'])

  return compras[(abs(z_scores) < 3) & (compras['valor_em_real'] > 0)]

compras_sem_valores_suspeitos(compras)


,nome_do_funcionario,item_comprado,valor_em_real,data_da_compra
0,Amanda Jones,Copy Paper,227.00,2024-12-01 18:05:04
1,Patrick Kavuma,Counselling Services,88.00,2024-01-18 18:26:17
2,Daljit Singh,Tablet,1299.00,2024-06-12 09:05:57
3,"NARKOTA, ATEE",GASOLINA NATURAL ASASIN 100,17.60,2024-03-07 15:11:16
4,Cheryl Vile,Computer and accessories,2.81,2024-03-20 10:48:04
...,...,...,...,...
4995,Mrs. D Nelson,Purchase Order,20.21,2024-07-02 21:51:20
4996,Mr. Javier Chavez,Não foram encontrados registros.,194.00,2024-03-07 12:43:20
4997,"DEWAN, VANITA",Bottle of water,0.49,2024-04-18 12:43:54
4998,LAURA JACKSON,Half Pavement,13168.50,2024-03-04 03:02:39


In [96]:
def compras_fora_de_horario(compras):
  compras['data_da_compra'] = pd.to_datetime(compras['data_da_compra'])
  return compras[(compras['data_da_compra'].dt.hour > 18) | (compras['data_da_compra'].dt.hour < 8)]


In [97]:
def organizar_por_servidor(compras):
  return compras.groupby('nome_do_funcionario').apply(
      lambda group: group[['item_comprado', 'valor_em_real']].to_dict(orient='records')
  ).to_dict()


def funcionario_mais_menos_vendeu(compras):
    vendas_por_funcionario = compras.groupby('nome_do_funcionario')['valor_em_real'].sum()

    funcionario_mais_vendeu = vendas_por_funcionario.idxmax()
    valor_mais_vendeu = vendas_por_funcionario.max()

    funcionario_menos_vendeu = vendas_por_funcionario.idxmin()
    valor_menos_vendeu = vendas_por_funcionario.min()

    return {
        "Funcionario mais vendeu": {
            "nome": funcionario_mais_vendeu,
            "valor_total_vendido": valor_mais_vendeu
        },
        "Funcionario menos vendeu": {
            "nome": funcionario_menos_vendeu,
            "valor_total_vendido": valor_menos_vendeu
        }
    }


In [117]:
def gerar_relatorio(compras):
  numero_compras_fora_do_horario = compras_fora_de_horario(compras).shape[0]
  numero_de_fraudes = verificar_valores_suspeitos(compras).shape[0]
  numero_compras_duplicadas = detectar_compras_duplicadas(compras).shape[0]

  compras_okays = compras_sem_valores_suspeitos(compras)

  item_mais_barato = compras_okays.loc[compras_okays['valor_em_real'].idxmin()]
  item_mais_caro = compras_okays.loc[compras_okays['valor_em_real'].idxmax()]

  melhores_funcionarios = funcionario_mais_menos_vendeu(compras_okays)

  return {
    "Quantidade de compras": compras.shape[0],
    "Quantidade de compras fora do horário": numero_compras_fora_do_horario,
    "Quantidade de compras suspeitas": numero_de_fraudes,
    "Quantidade de compras duplicadas": numero_compras_duplicadas,
    "Item mais barato": f"{item_mais_barato['item_comprado']} - Valor: {item_mais_barato['valor_em_real']}",
    "Item mais caro": f"{item_mais_caro['item_comprado']} - Valor: {item_mais_caro['valor_em_real']}",
    "Melhores funcionarios": melhores_funcionarios
}


In [118]:
gerar_relatorio(compras)

{'Quantidade de compras': 5000,
 'Quantidade de compras fora do horário': 1058,
 'Quantidade de compras suspeitas': 7,
 'Quantidade de compras duplicadas': 32,
 'Item mais barato': 'Government - Valor: 0.0155',
 'Item mais caro': '7000000,00 - Valor: 7000000.0',
 'Melhores funcionarios': {'Funcionario mais vendeu': {'nome': 'GREY, ROBERT',
   'valor_total_vendido': 7000000.0},
  'Funcionario menos vendeu': {'nome': 'KHANUM N. P.',
   'valor_total_vendido': 0.0155}}}